In [7]:
! pip install nltk
! pip install numpy
import random
import json
import pickle
import numpy as np
import nltk
! python -m nltk.downloader all
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
print('Setup done!')

zsh:1: command not found: pip
     |████████████████████████████████| 16.9 MB 8.6 MB/s 
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'numpy'

In [13]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('./input/intents.json').read())

words = []
topics = []
documents = []
ignore_letters = ['?', '!', '.', ',', ':', ';', '"', '\'']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        for word in word_list:
            words.append(word)
        documents.append((word_list, intent['topic']))
        if intent['topic'] not in topics:
            topics.append(intent['topic'])

words = [lemmatizer.lemmatize(word, 'v') for word in words if word not in ignore_letters]
words = sorted(set(words))
topics = sorted(set(topics))

pickle.dump(words, open('output/words.pk1', 'wb'))
pickle.dump(topics, open('output/topics.pk1', 'wb'))

training = []
output_empty = [0] * len(topics)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1 if word in word_patterns else 0)
    output_row = list(output_empty)
    output_row[topics.index(document[1])] = 1
    training.append([bag, output_row])

NameError: name 'WordNetLemmatizer' is not defined

In [12]:
training

NameError: name 'training' is not defined

In [11]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

NameError: name 'training' is not defined

In [ ]:
model = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    Dropout(.5),
    Dense(64, activation='relu'),
    Dropout(.5),
    Dense(len(train_y[0]), activation='softmax'),
])

sgd = SGD(lr=.01, decay=1e-6, momentum=.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('output/SlackBotNLP.model')
print("All set!")